In [1]:
import pandas_datareader.data as web
import datetime
import pandas as pd
import pandas_datareader as pdr
import numpy as np
import pickle
from sklearn.linear_model import LogisticRegression

In [2]:
class stock:
    def __init__(self,code,gs):
        self.code = code
        self.gs = gs
    def get_data(self,days):
        self.open_list = self.gs['Open'][-days-1:]
        self.volume_list = self.gs['Volume'][-days-1:]
        if self.open_list[-2] < self.open_list[-1]:
            self.increase = True
        else:
            self.increase = False

class x_y:
    def __init__(self,days):
        with open('stock_list_small.pickle','rb') as f:
            stock_list = pickle.load(f)
        pass_fail = []
        for stock in stock_list:
            try:
                stock.get_data(days)
                pass_fail.append(True)
            except:
                pass_fail.append(False)
        x_with_zeros= np.zeros(shape=(np.sum(pass_fail),days*2),dtype=np.int)
        y=[]
        idx=0
        for idx, s in enumerate(stock_list):
            if pass_fail[idx]:
                try:
                    x_with_zeros[idx,:] = list(list(s.open_list[:-1].values)+list(s.volume_list[:-1].values))
                    y.append(s.increase)
                except:
                    pass
                idx+=1
        x= np.zeros(shape=(len(y),days*2),dtype=np.int)
        zero_num=0
        for idx, x_ in enumerate(x_with_zeros):
            if np.sum(x_)==0:
                pass
            else:
                x[zero_num,:] = x_
                zero_num+=1
        self.x = x
        self.y = y
        self.days=days

class stock_bot:
    def __init__(self,x,y,days):
        self.org_x = x.copy()
        self.org_y = y.copy()
        self.days = days
        self.x = x.copy()
        self.y = y.copy()
    def set_org(self):
        self.x = self.org_x
        self.y = self.org_y
    def set_x_price_par(self,p):
        self.x[:,:self.days] = self.org_x[:,:self.days]*p
    def set_x_volume_par(self,p):
        self.x[:,self.days:] = self.org_x[:,self.days:]*p
    def ret_clf(self):
        return LogisticRegression(random_state=0).fit(self.x,self.y)

class stock_bot_final:
    def __init__(self,days,p1,p2):
        self.xy = x_y(days)
        self.sb = stock_bot(self.xy.x,self.xy.y,self.xy.days)
        self.sb.set_x_price_par(p1)
        self.sb.set_x_volume_par(p2)
    def ret_clf(self):
        self.clf= LogisticRegression(random_state=0).fit(self.sb.x,self.sb.y)
        return self.clf
    def validate(self):
        right=0
        wrong = 0 
        for idx, x_ in enumerate(self.xy.x):
            if(self.clf.predict(x_.reshape(1,-1))[0]==self.xy.y[idx]):
                right+=1
            else:
                wrong+=1
        print(right,wrong)        

In [3]:
sbf = stock_bot_final(10,0.5,0.1)

clf = sbf.ret_clf()

sbf.validate()

1009 834


C:\Users\vldb\anaconda3\envs\th\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [5]:
sbf = stock_bot_final(20,0.5,0.01)

clf = sbf.ret_clf()

sbf.validate()

1038 805


C:\Users\vldb\anaconda3\envs\th\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
clf.predict_prob()